In [1]:
# hide ssl warnings for this test.
import requests
requests.packages.urllib3.disable_warnings()

# Performing scripts with python-fmrest

This is a short example on how to perform scripts with python-fmrest.

## Import the module

In [2]:
import fmrest

## Create the server instance

In [3]:
fms = fmrest.Server('https://10.211.55.15',
                    user='admin',
                    password='admin',
                    database='Contacts',
                    layout='Demo',
                    verify_ssl=False
                   )

## Login

The login method obtains the access token.

In [4]:
fms.login()

'852a4bd3c64e58204c33af7c86f0d70dc52d04a6c4bc1210b37c'

## Setup scripts

You can setup scripts to run prerequest, presort, and after the action and sorting are executed. The script setup is passed to a python-fmrest method as an object that contains the types of script executions, followed by a list containing the script name and parameter.

In [5]:
scripts={
    'prerequest': ['name_of_script_to_run_prerequest', 'script_parameter'],
    'presort': ['name_of_script_to_run_presort', None], # parameter can also be None
    'after': ['name_of_script_to_run_after_actions', '1234'], #FMSDAPI expects all parameters to be string
}

You only need to specify the scripts you actually want to execute. So if you only have an after action, just build a scripts object with only the 'after' key.

## Call a standard method

Scripts are always executed as part of a standard request to the server. These requests are the usual `find()`, `create_record()`, `delete_record()`, `edit_record()`, `get_record()` methods the Server class exposes to you.

Let's make a find and then execute a script. The script being called contains an error on purpose, so that we can later read out the error number.

In [6]:
fms.find(
    query=[{'name': 'David'}],
    scripts={
        'after': ['testScriptWithError', None],
    }
)

<Foundset consumed_records=0 is_complete=False>

## Get the last script error and result

Via the `last_script_result` property, you can access both last error and script result for all scripts that were called.

In [7]:
fms.last_script_result

{'after': [3, '1']}

We see that we had 3 as last error, and our script result was '1'. The FMS Data API only returns strings, but error numbers are automatically converted to integers, for convenience. The script result, however, will always be a string or None, even if you exit your script in FM with a number or boolean.

## Another example

Let's do another call, this time with a script that takes a parameter and does not have any errors.
It will exit with `Exit Script[ Get(ScriptParameter) ]`, so essentially give us back what we feed in.

In [8]:
fms.find(
    query=[{'name': 'David'}],
    scripts={
        'prerequest': ['demoScript (id)', 'abc-1234'],
    }
)

<Foundset consumed_records=0 is_complete=False>

... and here is the result (error 0 means no error):

In [9]:
fms.last_script_result

{'prerequest': [0, 'abc-1234']}